# Battle of Neighborhoods--Best Way to Visit Tokyo

# Week1

# 1. Introduction of Problem and Background
Make a plan to visit Tokyo based on the analysis of the data of the venuse from the Foursquare around the top 9 sights suggested by Google 


# 2. Data

In [1]:
import re, json

from sklearn.cluster import KMeans
import requests
import folium
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from matplotlib import cm, colors

## 2.1 Scrapping Top 9 sights in Tokyo from Google

In [2]:
link = 'https://www.google.com/travel/things-to-do?g2lb=2502548%2C4258168%2C4260007%2C4270442%2C4274032%2C4291318%2C4305595%2C4306835%2C4309598%2C4317915%2C4328159%2C4329288%2C4333265%2C4357967%2C4366684%2C4366858%2C4367954%2C4369397%2C4370792%2C4373085%2C4270859%2C4284970%2C4291517%2C4316256%2C4356900&hl=en&gl=jp&un=1&otf=1&dest_mid=%2Fg%2F12lnhn10f&dest_state_type=main&dest_src=ts&tcfs=Eg4KDC9nLzEybG5objEwZg&sa=X&utm_campaign=sharing&utm_medium=link&utm_source=htls#ttdm=35.640148_139.789860_11&ttdmf=%25252Fm%25252F0132_x'
source_tokyo = requests.get(link).text
soup_tokyo = BeautifulSoup(source_tokyo, 'lxml')

In [3]:
script_coords = soup_tokyo.find('script', id='wiz_jd').previous_sibling

In [4]:
start = re.search('return ', script_coords.contents[0]).end()
print(start)

86


In [5]:
topsights9 = json.loads(script_coords.contents[0][start : -4])
df_tokyo_top9 = pd.DataFrame(index=range(9), columns=['name', 'latitude', 'longitude'])
for i in range(9):
  try:
    name = topsights9[0][0][0][-1][0][i][0][1]
    coord = topsights9[0][0][0][-1][0][i][0][-10]
  except TypeError: 
    name = topsights9[0][0][1][-1][0][i][0][1]
    coord = topsights9[0][0][1][-1][0][i][0][-10]
  if name == 'Sensō-ji': name = 'Senso-ji' # change the name of 'Sensō-ji'
  df_tokyo_top9.iloc[i, :] = [name] + coord
  print('Top {}: {}, {}'.format(i+1, name, coord))

Top 1: Tokyo Skytree, [35.7100627, 139.81070040000003]
Top 2: Senso-ji, [35.7147651, 139.7966553]
Top 3: Tokyo Tower, [35.6585805, 139.7454329]
Top 4: Meiji Jingu, [35.6763976, 139.6993259]
Top 5: Tokyo Disneyland, [35.6328964, 139.8803943]
Top 6: Odaiba, [35.62058340000001, 139.7805445]
Top 7: Ueno Park, [35.7154769, 139.7740551]
Top 8: Imperial Palace, [35.685175, 139.75279949999998]
Top 9: Shinjuku Gyoen National Garden, [35.685176299999995, 139.7100517]


In [6]:
df_tokyo_top9.to_csv("top9sights_tokyo.csv")
df_tokyo_top9

,name,latitude,longitude
0,Tokyo Skytree,35.7101,139.811
1,Senso-ji,35.7148,139.797
2,Tokyo Tower,35.6586,139.745
3,Meiji Jingu,35.6764,139.699
4,Tokyo Disneyland,35.6329,139.88
5,Odaiba,35.6206,139.781
6,Ueno Park,35.7155,139.774
7,Imperial Palace,35.6852,139.753
8,Shinjuku Gyoen National Garden,35.6852,139.71


### Vusualize the top 9 sights on map

In [7]:
coor_imperial_palace = df_tokyo_top9.iloc[7, 1:]
map_tokyo = folium.Map(location=coor_imperial_palace, width='80%', height='80%', zoom_start=12)
# add marker to map
for name, lat, lng in zip(df_tokyo_top9.loc[:, 'name'], df_tokyo_top9.loc[:, 'latitude'], df_tokyo_top9.loc[:, 'longitude']):
  label = '{}, Tokyo'.format(name)
  label = folium.Popup(label, parse_html=True, max_width=300)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map_tokyo)  
map_tokyo

#2.2 Top 100 venues around each sights from Foursquare

Define the Foursquare Credential and Version

In [20]:
CLIENT_ID = '****' # your Foursquare ID
CLIENT_SECRET = '****' # your Foursquare Secret
VERSION = '20200501' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ****
CLIENT_SECRET:****


Define a function to get the top venues for the top 9 sights

In [9]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']      
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'sightLatitude', 
                  'sightLongitude', 
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory'] 
    return(nearby_venues)

Get the venues within 700 meters around each sight

In [10]:
top9sights_venues = getNearbyVenues(df_tokyo_top9.name, df_tokyo_top9.latitude, df_tokyo_top9.longitude, radius=700)

Tokyo Skytree
Senso-ji
Tokyo Tower
Meiji Jingu
Tokyo Disneyland
Odaiba
Ueno Park
Imperial Palace
Shinjuku Gyoen National Garden


In [11]:
print('Shape: ', top9sights_venues.shape)
print("Nunber of the unique venue categories: ", len(top9sights_venues.VenueCategory.unique()))

Shape:  (773, 7)
Nunber of the unique venue categories:  179


In [12]:
# the counts of venues around each sight.
top9sights_venues.groupby('name').count()

,sightLatitude,sightLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
name,,,,,,
Imperial Palace,30,30,30,30,30,30
Meiji Jingu,66,66,66,66,66,66
Odaiba,80,80,80,80,80,80
Senso-ji,100,100,100,100,100,100
Shinjuku Gyoen National Garden,100,100,100,100,100,100
Tokyo Disneyland,100,100,100,100,100,100
Tokyo Skytree,100,100,100,100,100,100
Tokyo Tower,97,97,97,97,97,97
Ueno Park,100,100,100,100,100,100


In [13]:
top9sights_venues.head()

,name,sightLatitude,sightLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,Tokyo Skytree,35.710063,139.8107,Tokyo Skytree (東京スカイツリー),35.710054,139.810710,Monument / Landmark
1,Tokyo Skytree,35.710063,139.8107,Kitano Ace (北野エース),35.710220,139.809799,Grocery Store
2,Tokyo Skytree,35.710063,139.8107,Sumida Aquarium (すみだ水族館),35.710046,139.809639,Aquarium
3,Tokyo Skytree,35.710063,139.8107,Tokyo Skytree Tembo Galleria (東京スカイツリー天望回廊),35.710132,139.810730,Scenic Lookout
4,Tokyo Skytree,35.710063,139.8107,Moomin House Cafe (ムーミンハウスカフェ),35.710404,139.809349,Café


# 3 Top 10 Common Venue Categories and Clusters of 9 Sights

##  3.1 One-hot vectorize the venue categories 

In [14]:
venues_onehot = pd.get_dummies(top9sights_venues.iloc[:,-1])
venues_onehot.insert(0, top9sights_venues.columns[0], top9sights_venues.iloc[:,0])
print("shape: ",venues_onehot.shape)
venues_onehot.head()

shape:  (773, 180)


,name,ATM,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,BBQ Joint,...,Unagi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wagashi Place,Well,Wine Bar,Yakitori Restaurant,Yoshoku Restaurant,Zoo,Zoo Exhibit
0,Tokyo Skytree,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Tokyo Skytree,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Tokyo Skytree,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Tokyo Skytree,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Tokyo Skytree,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Groupby 'name'

In [15]:
top9sights_venues_grouped = venues_onehot.groupby(by='name').mean().reset_index()
top9sights_venues_grouped

,name,ATM,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,BBQ Joint,...,Unagi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wagashi Place,Well,Wine Bar,Yakitori Restaurant,Yoshoku Restaurant,Zoo,Zoo Exhibit
0,Imperial Palace,0.00,0.000000,0.00,0.000000,0.066667,0.00,0.00,0.0000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00
1,Meiji Jingu,0.00,0.000000,0.00,0.015152,0.000000,0.00,0.00,0.0000,0.030303,...,0.000000,0.00,0.00,0.00,0.015152,0.00,0.000000,0.00,0.00,0.00
2,Odaiba,0.00,0.000000,0.00,0.000000,0.012500,0.00,0.00,0.0125,0.012500,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.025000,0.00,0.00,0.00
3,Senso-ji,0.00,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.0000,0.050000,...,0.040000,0.00,0.01,0.05,0.000000,0.00,0.000000,0.04,0.00,0.00
4,Shinjuku Gyoen National Garden,0.01,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.0000,0.030000,...,0.000000,0.01,0.01,0.00,0.000000,0.00,0.000000,0.01,0.00,0.00
5,Tokyo Disneyland,0.00,0.020000,0.00,0.000000,0.000000,0.00,0.01,0.0000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.01,0.00,0.00
6,Tokyo Skytree,0.00,0.010000,0.01,0.000000,0.000000,0.01,0.00,0.0000,0.040000,...,0.010000,0.00,0.00,0.00,0.000000,0.02,0.010000,0.01,0.00,0.00
7,Tokyo Tower,0.00,0.020619,0.00,0.000000,0.000000,0.00,0.00,0.0000,0.010309,...,0.010309,0.00,0.00,0.00,0.000000,0.00,0.020619,0.00,0.00,0.00
8,Ueno Park,0.00,0.010000,0.00,0.010000,0.050000,0.00,0.00,0.0000,0.010000,...,0.020000,0.01,0.00,0.04,0.000000,0.00,0.000000,0.01,0.01,0.11


## 3.2 Top 10 Common Venues for each sight

In [16]:
# define the function to return the top venues
def return_most_common_venues(row, num_top_venues):
    row_sorted = row.sort_values(ascending=False)
    return row_sorted.index.values[0:num_top_venues]

# make the columns
n_top = 10
indicator = ['st', 'nd', 'rd']
cols = ['name']
for idx in range(n_top):
  #tmp_col = '{}{}_Most_Common_Venue'
  tmp_col = '{}{}_MCV'
  try: new_col = tmp_col.format(idx+1, indicator[idx])
  except: new_col = tmp_col.format(idx+1, 'th')
  cols.append(new_col)

# get the top 10 common venues
top_venue = pd.DataFrame(columns=cols)
top_venue.iloc[:, 0] = top9sights_venues_grouped.iloc[:, 0]
# add top venues for each row
for idx in range(len(top9sights_venues_grouped)):
  top_venue.iloc[idx, 1:] = return_most_common_venues(
                                          top9sights_venues_grouped.iloc[idx, 1:], n_top)

top_venue

,name,1st_MCV,2nd_MCV,3rd_MCV,4th_MCV,5th_MCV,6th_MCV,7th_MCV,8th_MCV,9th_MCV,10th_MCV
0,Imperial Palace,Historic Site,Art Museum,Bridge,Palace,Library,Track,Plaza,Castle,Garden,Park
1,Meiji Jingu,Café,Italian Restaurant,Convenience Store,Japanese Restaurant,Rock Club,Park,Japanese Curry Restaurant,Indian Restaurant,Udon Restaurant,BBQ Joint
2,Odaiba,Coffee Shop,Plaza,Convenience Store,Exhibit,Japanese Restaurant,Pier,Gift Shop,Bar,Shopping Mall,Discount Store
3,Senso-ji,Japanese Restaurant,BBQ Joint,Soba Restaurant,Wagashi Place,Café,Coffee Shop,Dessert Shop,Unagi Restaurant,Yoshoku Restaurant,Hotel
4,Shinjuku Gyoen National Garden,Café,Ramen Restaurant,Japanese Restaurant,Garden,Coffee Shop,Bar,Burger Joint,Indian Restaurant,Theater,Chinese Restaurant
5,Tokyo Disneyland,Theme Park Ride / Attraction,Theme Park,Café,Gift Shop,Restaurant,Hotel,Snack Place,Japanese Restaurant,Lounge,Theme Restaurant
6,Tokyo Skytree,Convenience Store,Café,Ramen Restaurant,BBQ Joint,Scenic Lookout,Japanese Restaurant,Coffee Shop,Grocery Store,Gift Shop,Chinese Restaurant
7,Tokyo Tower,Chinese Restaurant,Convenience Store,Park,Japanese Restaurant,Italian Restaurant,Bakery,Pizza Place,Ramen Restaurant,Steakhouse,Grocery Store
8,Ueno Park,Zoo Exhibit,Sake Bar,History Museum,Art Museum,Wagashi Place,Coffee Shop,Chinese Restaurant,Historic Site,Japanese Restaurant,Museum


## 3.3 Cluster the top 9 sights to 5 cluster

Run k-means to cluster the neighborhoods into 4 clusters

In [17]:
k5 = 5
tokyo_cluster5 = top9sights_venues_grouped.drop('name', axis=1)
kmeans5 = KMeans(n_clusters=k5, random_state=42).fit(tokyo_cluster5)
kmeans5.labels_[:10]

array([2, 1, 3, 1, 1, 0, 3, 3, 4], dtype=int32)

In [18]:
df_tokyo_final5 = df_tokyo_top9.merge(top_venue, on='name')
df_tokyo_final5.insert(loc=3, column='ClusterLabel', value=kmeans5.labels_)
df_tokyo_final5

,name,latitude,longitude,ClusterLabel,1st_MCV,2nd_MCV,3rd_MCV,4th_MCV,5th_MCV,6th_MCV,7th_MCV,8th_MCV,9th_MCV,10th_MCV
0,Tokyo Skytree,35.7101,139.811,2,Convenience Store,Café,Ramen Restaurant,BBQ Joint,Scenic Lookout,Japanese Restaurant,Coffee Shop,Grocery Store,Gift Shop,Chinese Restaurant
1,Senso-ji,35.7148,139.797,1,Japanese Restaurant,BBQ Joint,Soba Restaurant,Wagashi Place,Café,Coffee Shop,Dessert Shop,Unagi Restaurant,Yoshoku Restaurant,Hotel
2,Tokyo Tower,35.6586,139.745,3,Chinese Restaurant,Convenience Store,Park,Japanese Restaurant,Italian Restaurant,Bakery,Pizza Place,Ramen Restaurant,Steakhouse,Grocery Store
3,Meiji Jingu,35.6764,139.699,1,Café,Italian Restaurant,Convenience Store,Japanese Restaurant,Rock Club,Park,Japanese Curry Restaurant,Indian Restaurant,Udon Restaurant,BBQ Joint
4,Tokyo Disneyland,35.6329,139.88,1,Theme Park Ride / Attraction,Theme Park,Café,Gift Shop,Restaurant,Hotel,Snack Place,Japanese Restaurant,Lounge,Theme Restaurant
5,Odaiba,35.6206,139.781,0,Coffee Shop,Plaza,Convenience Store,Exhibit,Japanese Restaurant,Pier,Gift Shop,Bar,Shopping Mall,Discount Store
6,Ueno Park,35.7155,139.774,3,Zoo Exhibit,Sake Bar,History Museum,Art Museum,Wagashi Place,Coffee Shop,Chinese Restaurant,Historic Site,Japanese Restaurant,Museum
7,Imperial Palace,35.6852,139.753,3,Historic Site,Art Museum,Bridge,Palace,Library,Track,Plaza,Castle,Garden,Park
8,Shinjuku Gyoen National Garden,35.6852,139.71,4,Café,Ramen Restaurant,Japanese Restaurant,Garden,Coffee Shop,Bar,Burger Joint,Indian Restaurant,Theater,Chinese Restaurant


## 3.4 Visualize the 5 clusters on the map

In [19]:
map_tokyo_clusters5 = folium.Map(location=coor_imperial_palace, 
                                       width='100%', height='100%', zoom_start=13)
# set color scheme for the clusters
colors_array = cm.rainbow(np.linspace(0, 0.1, k5))
rainbow = [colors.rgb2hex(i) for i in colors_array]

colors_array2 = cm.rainbow(np.linspace(0, 1, k5))
rainbow2 = [colors.rgb2hex(i) for i in colors_array2]

colors_icon = ['red', 'blue', 'green', 'purple', 'orange']

# add marker to map
for lat, lng, name, cluster in zip(df_tokyo_final5.latitude, 
                                                        df_tokyo_final5.longitude, 
                                                        df_tokyo_final5.name, 
                                                        df_tokyo_final5.ClusterLabel):
  label = '{}; Cluster {}'.format(name, cluster)
  label = folium.Popup(label, parse_html=True, max_width=50, show=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      #popup=label,
      color=rainbow[cluster],
      fill=True,
      fill_color=rainbow2[cluster],
      fill_opacity=0.7,
      parse_html=False).add_to(map_tokyo_clusters5) 
  folium.Marker([lat, lng], popup=label, icon=folium.Icon(color=colors_icon[cluster])).add_to(map_tokyo_clusters5) 

map_tokyo_clusters5